## Imports

In [1]:
import argparse
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from hops import hdfs
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import tensorflow as tf

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3250,application_1513605045578_0495,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Constants

In [2]:
project_path = "/Projects/" + hdfs.project_name()

TRAIN_FEATURES_PATH = project_path + "/HAR_Dataset/cleaned_data/train/features/x_train.csv"
TRAIN_LABELS_PATH = project_path + "/HAR_Dataset/cleaned_data/train/labels/y_train.csv"
TEST_FEATURES_PATH = project_path + "/HAR_Dataset/cleaned_data/test/features/x_test.csv"
TEST_LABELS_PATH = project_path + "/HAR_Dataset/cleaned_data/test/labels/y_test.csv"

OUTPUT_TRAIN_FEATURES_PATH = project_path + "/HAR_Dataset/cleaned_data_parallel/train/features"
OUTPUT_TRAIN_LABELS_PATH = project_path + "/HAR_Dataset/cleaned_data_parallel/train/labels"
OUTPUT_TEST_FEATURES_PATH = project_path + "/HAR_Dataset/cleaned_data_parallel/test/features"
OUTPUT_TEST_LABELS_PATH = project_path + "/HAR_Dataset/cleaned_data_parallel/test/labels"

sc = spark.sparkContext
sql = SQLContext(sc)

schema_label = (StructType().add('y', IntegerType(), True)) 

## Read Data

In [3]:
def read_raw_data():
    seq_features_train_raw = sc.textFile(TRAIN_FEATURES_PATH).zipWithIndex()
    seq_labels_train_raw = sc.textFile(TRAIN_LABELS_PATH).zipWithIndex()
    seq_features_test_raw = sc.textFile(TEST_FEATURES_PATH).zipWithIndex()
    seq_labels_test_raw = sc.textFile(TEST_LABELS_PATH).zipWithIndex()
    return seq_features_train_raw,seq_labels_train_raw, seq_features_test_raw, seq_labels_test_raw

### Read Raw Data

In [4]:
seq_features_train_raw, seq_labels_train_raw,seq_features_test_raw,seq_labels_test_raw = read_raw_data()

### Filter Null

In [5]:
def filter1(i,y):
    if int(y) != 6:
        return True
    else:
        return False
indices = seq_labels_train_raw.filter(lambda (y,i): not filter1(i,y)).map(lambda (y,i): i).collect()
seq_labels_train_raw = seq_labels_train_raw.filter(lambda (y,i): filter1(i,y))

In [6]:
indices2 = seq_labels_test_raw.filter(lambda (y,i): not filter1(i,y)).map(lambda (y,i): i).collect()
seq_labels_test_raw = seq_labels_test_raw.filter(lambda (y,i): filter1(i,y))

In [7]:
seq_features_train_raw = seq_features_train_raw.filter(lambda (x,i): i not in indices)
seq_features_test_raw = seq_features_test_raw.filter(lambda (x,i): i not in indices2)

In [8]:
s1 = seq_features_train_raw.count()
s2 = seq_features_test_raw.count()
s3 = seq_labels_train_raw.count()
s4 = seq_labels_test_raw.count()

print("{0},{1},{2},{3}".format(s1,s2,s3,s4))

451200,112743,451200,112743

### Save Cleaned Data

In [9]:
seq_features_train_raw.map(lambda (x,i): x).saveAsTextFile(OUTPUT_TRAIN_FEATURES_PATH)
seq_labels_train_raw.map(lambda (y,i): y).saveAsTextFile(OUTPUT_TRAIN_LABELS_PATH)
seq_features_test_raw.map(lambda (x,i): x).saveAsTextFile(OUTPUT_TEST_FEATURES_PATH)
seq_labels_test_raw.map(lambda (y,i): y).saveAsTextFile(OUTPUT_TEST_LABELS_PATH)